In [1]:
import ipywidgets as widgets
import time
from pyscf import gto, scf, mp, fci, ci, cc, dft

#create UI elements
molecules = widgets.Dropdown(
    description = 'Molecule:',
    options     = ['H2', 'HF', 'O2', 'N2'],
    value       = 'H2'
)
display(molecules)

bond_scale = widgets.FloatSlider(
    description = 'Scale bond: ',
    value = 1.0,
    min = 0.9,
    max = 1.1,
    step = 0.01,
    readout = True,
    readout_format = '.2f'
)
display(bond_scale)

basis_sets = widgets.Dropdown(
    description = 'Basis set:',
    options = ['STO-3G', '3-21G', '6-31G', '6-31+G', '6-31+G*', '6-311G'],
    value = 'STO-3G'
)
display(basis_sets)

methods = widgets.Dropdown(
    description = 'Method:',
    options = ['Hartree-Fock', 'MP2', 'FCI', 'CISD', 'CCSD', 'DFT: LDA', 'DFT: BLYP', 'DFT: B3LYP'],
    value = 'Hartree-Fock'
)
display(methods)

raw_output = widgets.Checkbox(
    description = 'Show raw output:', 
    value = False
)
display(raw_output)

run = widgets.Button(
    description  = 'Run',
    button_style = 'success',
    tooltip      = 'Run calculation',
    icon         = 'check'
)
display(run)

clear = widgets.Button(
    description  = 'Clear output',
    button_style = 'warning',
    tooltip      = 'Clear output',
    icon         = 'remove'
)
display(clear)

output = widgets.Output()
display(output)

#function to run the actual calculation
def run_calculation(b):
    
    #output what we are going to run
    with output:
        print("{:30s}: {:}".format('Molecule',             molecules.value))
        print("{:30s}: {:.2f}".format('Bond-length scale', bond_scale.value))
        print("{:30s}: {:}".format('Basis set',            basis_sets.value))
        print("{:30s}: {:}".format('Method',               methods.value))
    
    #define molecule parameters
    bond_lengths = {'H2': 1.4192, 'HF': 0.929, 'O2': 2.2828, 'N2': 2.0787}
    geometries = {'H2': 'H 0 0 0; H 0 0 {}'.format(bond_lengths['H2'] * bond_scale.value),
                  'HF': 'H 0 0 0; F 0 0 {}'.format(bond_lengths['HF'] * bond_scale.value),
                  'O2': 'O 0 0 0; O 0 0 {}'.format(bond_lengths['O2'] * bond_scale.value),
                  'N2': 'N 0 0 0; N 0 0 {}'.format(bond_lengths['N2'] * bond_scale.value)}
    spins      = {'H2': 0, 'HF': 0, 'O2': 2, 'N2': 0}
    
    #start a timer
    start_time = time.time()
    
    #build the molecule and set the verbosity level
    mol = gto.M(atom    = geometries[molecules.value],
                basis   = basis_sets.value,
                spin    = spins[molecules.value],
                verbose = 0)
    if raw_output.value:
        mol.verbose = 5
    
    #output some information on the calculation
    with output:
        print("{:30s}: {:}".format('Number of atoms',               mol.natm))
        print("{:30s}: {:}".format('Number of electrons',           mol.nelectron))
        print("{:30s}: {:}".format('Number of gaussian functions',  mol.nbas))
        print("{:30s}: {:}".format('Number of gaussian primitives', sum([i[2] for i in mol._bas])))
    
    #run the specified calculations
    if methods.value == 'Hartree-Fock':
        hf_calc = scf.HF(mol)
        res = hf_calc.scf()
        with output:
            print('{:30s}: {:.4f} Ha'.format('HF energy', res))
    
    elif methods.value == 'MP2':
        mp2_calc = mp.MP2(scf.HF(mol)).as_scanner()
        res = mp2_calc(mol)
        with output:
            print('{:30s}: {:.4f} Ha'.format('MP2 energy', res))
            
    elif methods.value == 'FCI':
        
        if molecules.value != 'H2' and molecules.value != 'HF':
            with output:
                print("Error: FCI is limited to H2 and HF")
                return
            
        if (basis_sets.value != 'STO-3G' and basis_sets.value != '3-21G' and 
            basis_sets.value != '6-31G' and basis_sets.value != '6-311G'):
            with output:
                print("Error: FCI is limited to STO-3G, 3-21G and 6-31G basis sets")
                return
        
        if molecules.value == 'HF' and basis_sets.value == '6-311G':
            with output:
                print("Warning: This calclation can take a while. Please be patient...")
                return
        
        hf_calc = scf.HF(mol)
        hf_calc.kernel()
        ci_calc = fci.FCI(mol, hf_calc.mo_coeff)
        res = ci_calc.kernel()
        with output:
            print('{:30s}: {:.4f} Ha'.format('FCI energy', res[0]))
            
    elif methods.value == 'CISD':
        ci_calc = ci.CISD(scf.HF(mol)).as_scanner()
        res = ci_calc(mol)
        with output:
            print('{:30s}: {:.4f} Ha'.format('CISD energy', res))
    
    elif methods.value == 'CCSD':
        cc_calc = cc.CCSD(scf.HF(mol)).as_scanner()
        res = cc_calc(mol)
        with output:
            print('{:30s}: {:.4f} Ha'.format('CCSD energy', res))
    
    elif methods.value.startswith('DFT:'):
        functional = methods.value.split()[1]
        dft_calc = dft.KS(mol)
        dft_calc.xc = functional
        res = dft_calc.kernel()
        with output:
            print('{:30s}: {:.4f} Ha'.format('{} energy'.format(methods.value), res))
        
    #measure and output elapsed time 
    elapsed_time = time.time() - start_time
    with output:
        print('{:30s}: {:.2f} sec'.format('Elapsed time', elapsed_time))
        print('-------------------------------------------------------')

#function to clear the output
def clear_output(b):
    output.clear_output()
    
#hook up the functions to the buttons
run.on_click(run_calculation)
clear.on_click(clear_output)

Dropdown(description='Molecule:', options=('H2', 'HF', 'O2', 'N2'), value='H2')

FloatSlider(value=1.0, description='Scale bond: ', max=1.1, min=0.9, step=0.01)

Dropdown(description='Basis set:', options=('STO-3G', '3-21G', '6-31G', '6-31+G', '6-31+G*', '6-311G'), value=…

Dropdown(description='Method:', options=('Hartree-Fock', 'MP2', 'FCI', 'CISD', 'CCSD', 'DFT: LDA', 'DFT: BLYP'…

Checkbox(value=False, description='Show raw output:')

Button(button_style='success', description='Run', icon='check', style=ButtonStyle(), tooltip='Run calculation'…

Button(button_style='warning', description='Clear output', icon='remove', style=ButtonStyle(), tooltip='Clear …

Output()